In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
import re
import numpy as np

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
ticket_set = pd.read_csv('/content/drive/My Drive/Trictractrip/data/ticket_data.csv') #import
city_set = pd.read_csv('/content/drive/My Drive/Trictractrip/data/cities.csv', usecols = ['id','unique_name','latitude','longitude']) #import
providers_set = pd.read_csv('/content/drive/My Drive/Trictractrip/data/providers.csv') #import

La durée n'est pas présente, il faut la calculer à partir des dates de départ et d'arrivées.
Plutot que de la recalculer à chaque fois, je l'intègre à la vue



In [ ]:
ticket_set.insert(6,"time_in_min",0)
ticket_set.head(5) #check

,id,company,o_station,d_station,departure_ts,arrival_ts,time_in_min,price_in_cents,search_ts,middle_stations,other_companies,o_city,d_city
0,6795025,8385,NaN,NaN,2017-10-13 14:00:00+00,2017-10-13 20:10:00+00,0,4550,2017-10-01 00:13:31.327+00,NaN,NaN,611,542
1,6795026,9,63.0,1044.0,2017-10-13 13:05:00+00,2017-10-14 06:55:00+00,0,1450,2017-10-01 00:13:35.773+00,"{149,418}",{13},611,542
2,6795027,8377,5905.0,6495.0,2017-10-13 13:27:00+00,2017-10-14 21:24:00+00,0,7400,2017-10-01 00:13:40.212+00,"{798,798,6794,6246}","{8377,8376}",611,542
3,6795028,8377,5905.0,6495.0,2017-10-13 13:27:00+00,2017-10-14 11:02:00+00,0,13500,2017-10-01 00:13:40.213+00,"{798,798,6794,6246}","{8377,8376}",611,542
4,6795029,8381,5905.0,6495.0,2017-10-13 21:46:00+00,2017-10-14 19:32:00+00,0,7710,2017-10-01 00:13:40.213+00,"{5983,5983}",{8380},611,542


In [ ]:
dep_time = pd.to_datetime(ticket_set["departure_ts"])
arr_time = pd.to_datetime(ticket_set["arrival_ts"])
ticket_set["time_in_min"]=(arr_time-dep_time).dt.total_seconds()/60
ticket_set.sort_values(by=['time_in_min'])


,id,company,o_station,d_station,departure_ts,arrival_ts,time_in_min,price_in_cents,search_ts,middle_stations,other_companies,o_city,d_city
25912,6820937,8385,NaN,NaN,2017-10-13 17:20:00+00,2017-10-13 17:40:00+00,20.0,1100,2017-10-08 00:52:42.171+00,NaN,NaN,628,611
25813,6820838,8385,NaN,NaN,2017-10-13 17:20:00+00,2017-10-13 17:40:00+00,20.0,1100,2017-10-08 00:50:15.469+00,NaN,NaN,628,611
25846,6820871,8385,NaN,NaN,2017-10-13 17:20:00+00,2017-10-13 17:40:00+00,20.0,1100,2017-10-08 00:51:07.984+00,NaN,NaN,628,611
25881,6820904,8385,NaN,NaN,2017-10-13 17:20:00+00,2017-10-13 17:40:00+00,20.0,1100,2017-10-08 00:51:37.091+00,NaN,NaN,628,611
25770,6820804,8385,NaN,NaN,2017-10-13 17:20:00+00,2017-10-13 17:40:00+00,20.0,1100,2017-10-08 00:43:33.185+00,NaN,NaN,628,611
...,...,...,...,...,...,...,...,...,...,...,...,...,...
47856,6842881,8371,1044.0,10748.0,2017-12-23 11:00:00+00,2018-01-07 11:00:00+00,21600.0,4800,2017-10-13 07:14:20.422+00,"{496,496}",{8372},542,7507
47858,6842883,8371,1044.0,10748.0,2017-12-23 11:00:00+00,2018-01-07 11:00:00+00,21600.0,4800,2017-10-13 07:16:01.076+00,"{496,496}",{8372},542,7507
51070,6846095,8371,1044.0,10748.0,2017-12-22 11:00:00+00,2018-01-06 11:00:00+00,21600.0,4800,2017-10-14 08:27:16.4+00,"{496,496}",{8372},542,7507
5176,6800201,8372,860.0,396.0,2017-10-27 23:59:00+00,2017-11-17 06:40:00+00,29201.0,3390,2017-10-02 20:39:10.75+00,"{10760,279}",{13},126,453


Nous avons désormais la durée du trajet.

Il est désormais possible d'extraire les infos intéressantes type prix min, moyen et max, durée min/max/moyenne par trajet

In [ ]:
#En fonction du temps ou du prix

def trajet_time_info(time):
  depart    = ticket_set.loc[ticket_set['time_in_min'] == time, 'o_city'].map(city_set.set_index('id')['unique_name']).str.title().iloc[0]
  arrive    = ticket_set.loc[ticket_set['time_in_min'] == time, 'd_city'].map(city_set.set_index('id')['unique_name']).str.title().iloc[0]
  company   = ticket_set.loc[ticket_set['time_in_min'] == time, 'company'].map(providers_set.set_index('id')['fullname']).str.title().iloc[0]
  transport = ticket_set.loc[ticket_set['time_in_min'] == time, 'company'].map(providers_set.set_index('id')['transport_type']).iloc[0]

  t_unit = "dure "+ str(round(time/(60),1)) + " heures"
  if time <60:
    t_unit = "dure "+ str(round(time)) + " minutes"
  if time > 60*24:  
    t_unit= "dure "+ str(round(time/(60*24),1)) + " jours"  
  return t_unit + " et part de " + str(depart) + " à destination de " + str(arrive) + " de "+ str(company) + "(" +str(transport)+ ")."

def trajet_price_info(price):
  depart = ticket_set.loc[ticket_set['price_in_cents'] == price, 'o_city'].map(city_set.set_index('id')['unique_name']).str.title().iloc[0]
  arrive = ticket_set.loc[ticket_set['price_in_cents'] == price, 'd_city'].map(city_set.set_index('id')['unique_name']).str.title().iloc[0]
  company   = ticket_set.loc[ticket_set['price_in_cents'] == price, 'company'].map(providers_set.set_index('id')['fullname']).str.title().iloc[0]
  transport = ticket_set.loc[ticket_set['price_in_cents'] == price, 'company'].map(providers_set.set_index('id')['transport_type']).iloc[0]

  
  rep = "coûte "+ str(round(price/100,1)) + " € et part de " + str(depart) + " à destination de " + str(arrive) + " de "+ str(company) + "(" +str(transport)+ ")."
  return rep

In [ ]:
print("Le trajet le plus long "   + trajet_time_info(ticket_set["time_in_min"].max()) + " \n" +
      "Le trajet le plus court "  + trajet_time_info(ticket_set["time_in_min"].min()) + " \n" +
      "Le trajet median "         + trajet_time_info(ticket_set["time_in_min"].median())+ " (pour l'exemple) \n" +
      "Un trajet dure en moyenne "+ str(round(ticket_set["time_in_min"].mean()/60,1))  + " heures\n" )

print("Le trajet le plus cher "  +  trajet_price_info(ticket_set["price_in_cents"].max())    + "\n" +
      "Le trajet le moins cher " +  trajet_price_info(ticket_set["price_in_cents"].min())    + "\n" +
      "Le trajet médian "        +  trajet_price_info(ticket_set["price_in_cents"].median()) + " (pour l'exemple) \n" +

      "Un trajet coûte en moyenne "    +  str(round(ticket_set["price_in_cents"].mean()/100,1))  + " €. \n") # je présume que les prix sont en €

Le trajet le plus long dure 20.5 jours et part de Bordeaux à destination de Lille de Eurolines(bus). 
Le trajet le plus court dure 20 minutes et part de Paris à destination de Orleans de Blablacar(carpooling). 
Le trajet median dure 4.8 heures et part de Paris à destination de Rennes de Blablacar(carpooling). (pour l'exemple) 
Un trajet dure en moyenne 7.1 heures

Le trajet le plus cher coûte 385.5 € et part de London à destination de Bordeaux de Eurostar(train).
Le trajet le moins cher coûte 3.0 € et part de Auxerre à destination de Clamecy de Blablacar(carpooling).
Le trajet médian coûte 33.5 € et part de Paris à destination de Rennes de Blablacar(carpooling). (pour l'exemple) 
Un trajet coûte en moyenne 43.8 €. 



In [ ]:
#en fonction de l'origine et de la destination


def trajet_info(depart, arrive):
  def select_cf_price(tickets):
    print("Pour les tickets les moins chers de "+depart+ " à "+ arrive)
    get_data(tickets[ticket_set.price_in_cents.eq(tickets["price_in_cents"].min()) ].head(5))
    print("Pour les tickets les plus chers de "+depart+ " à "+ arrive)
    get_data(tickets[ticket_set.price_in_cents.eq(tickets["price_in_cents"].max()) ].head(5))
    print("Pour les tickets au prix médians de "+depart+ " à "+ arrive)
    get_data(tickets[ticket_set.price_in_cents.eq(tickets["price_in_cents"].median()) ].head(5))

  def get_data(tickets):
    company   = tickets['company'].map(providers_set.set_index('id')['fullname'])
    transport = tickets['company'].map(providers_set.set_index('id')['transport_type']).rename('transport_type')
    corresp   = tickets['middle_stations'].fillna("non").apply(lambda x: len(re.findall("\d+", x)) ).rename('correspondances')

    data = pd.concat([tickets['id'],tickets['price_in_cents'], company, transport,corresp], axis=1)
    print(str(data))

  ref_dep = int( city_set.loc[city_set['unique_name'] == depart, 'id'] )
  ref_arr = int( city_set.loc[city_set['unique_name'] == arrive, 'id'] )

  ticket_depart = ticket_set[ticket_set.o_city.eq(ref_dep) & ticket_set.d_city.eq(ref_arr)]

  select_cf_price(ticket_depart)
  
######  infos ici  ######
###### v v v v v v ######

trajet_info("paris","rennes")

###### ʌ ʌ ʌ ʌ ʌ ʌ ######

Pour les tickets les moins chers de paris à rennes
            id  price_in_cents   company transport_type  correspondances
3885   6798910            1000  Isilines            bus                2
3886   6798911            1000  Isilines            bus                2
64193  6859218            1000  Isilines            bus                2
64194  6859219            1000  Isilines            bus                2
Pour les tickets les plus chers de paris à rennes
            id  price_in_cents company transport_type  correspondances
17088  6812086           11650     TGV          train                2
Pour les tickets au prix médians de paris à rennes
          id  price_in_cents    company transport_type  correspondances
54   6795260            2650  Blablacar     carpooling                0
222  6795246            2650  Blablacar     carpooling                0
223  6795247            2650  Blablacar     carpooling                0
226  6795250            2650  Blablacar     carpoolin

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if __name__ == '__main__':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  # This is added back by InteractiveShellApp.init_path()


> différence de prix moyen et durée selon le train, le bus et le covoit selon la distance du trajet (0-200km, 201-800km, 800-2000km, 2000+km)

In [ ]:
import math

# our calculer les distances long-lat

def distance(origin, destination):
    lat1, lon1 = origin
    lat2, lon2 = destination
    radius = 6371 # km

    dlat = math.radians(lat2-lat1)
    dlon = math.radians(lon2-lon1)
    a = math.sin(dlat/2) * math.sin(dlat/2) + math.cos(math.radians(lat1)) \
        * math.cos(math.radians(lat2)) * math.sin(dlon/2) * math.sin(dlon/2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    d = radius * c

    return d


In [ ]:
# Le calcul de la distance à partir de llat-long est assez lourd, pour l'exemple j'utilise un set raccourci à 3000 rangées
ticket_short = pd.read_csv('/content/drive/My Drive/Trictractrip/data/ticket_data.csv', nrows=3000) #import
ticket_short.insert(3,"distance",0)
ticket_short.insert(3,"transport",0)

In [ ]:
def distance_cities(depart,arrive):
  depart_lat  = float ( city_set.loc[city_set['unique_name'] == depart, 'latitude' ])
  depart_long = float ( city_set.loc[city_set['unique_name'] == depart, 'longitude'])
  arrive_lat  = float ( city_set.loc[city_set['unique_name'] == arrive, 'latitude' ])
  arrive_long = float ( city_set.loc[city_set['unique_name'] == arrive, 'longitude'])
  return distance([ depart_lat , depart_long ], [ arrive_lat , arrive_long ]) 

def distance_cities_id(depart,arrive):
  dep_nom = ticket_set.loc[ticket_set['o_city'] == depart, 'o_city'].map(city_set.set_index('id')['unique_name']).iloc[0]
  arr_nom = ticket_set.loc[ticket_set['d_city'] == arrive, 'd_city'].map(city_set.set_index('id')['unique_name']).iloc[0]
  return distance_cities(dep_nom, arr_nom) 

def transport_mode(id):
  transport = str(ticket_set.loc[ticket_set['id']==id,"company"].map(providers_set.set_index('id')['transport_type']).iloc[0] )
  return transport

ticket_short["distance"]  = ticket_short.apply(lambda row: distance_cities_id( row["o_city"] , row["d_city"] ), axis=1 )
ticket_short["transport"] = ticket_short.apply(lambda row: transport_mode( row["id"] ), axis=1 )
print( str(ticket_short.distance.unique()) + "\n transports: \n"+ str(ticket_short.transport.unique()))#check

In [ ]:
def select_cf_dist_tran(tickets):

  def get_data(tickets):
    company   = tickets['company'].map(providers_set.set_index('id')['fullname'])
    corresp   = tickets['middle_stations'].fillna("non").apply(lambda x: len(re.findall("\d+", x)) ).rename('correspondances')
    depart    = tickets['o_city'].map(city_set.set_index('id')['unique_name']).str.title()
    destin    = tickets['d_city'].map(city_set.set_index('id')['unique_name']).str.title()

    data = pd.concat([tickets['id'],depart,destin,tickets['price_in_cents'], company, corresp], axis=1)
    print(str(data))


  print("les tickets les moins chers sont:")
  get_data(tickets[ticket_set.price_in_cents.eq(tickets["price_in_cents"].min()) ].head(5))
  print(" les tickets les plus chers sont:")
  get_data(tickets[ticket_set.price_in_cents.eq(tickets["price_in_cents"].max()) ].head(5))
  print(" les tickets au prix médians chers sont:")
  get_data(tickets[ticket_set.price_in_cents.eq(tickets["price_in_cents"].median()) ].head(5))


dist_group=[[0,200],[200,800],[800,2000],[2000,410000]] #410000km perimetre de la Terre, safe comme valeur max
tran_group=['carpooling', 'bus', 'train']

for dist in dist_group:
  print(str(dist[0])+" - "+str(dist[1]))
  df= ticket_short.query(str(dist[0])+ '<= distance <=' +str(dist[1]))
  for tran in tran_group:
    df_tran=df[df['transport'].str.match(tran)]
    print("les voyages en "+ tran+" de "+str(dist[0])+" à "+str(dist[1])+"km :\n")

    select_cf_dist_tran(df_tran)

0 - 200
les voyages en carpooling de 0 à 200km :

les tickets les moins chers sont:
           id                   o_city  ...    company  correspondances
2578  6797603  Beauvais-Aeroport-Paris  ...  Blablacar                0
2581  6797606  Beauvais-Aeroport-Paris  ...  Blablacar                0
2893  6797918                   Rennes  ...  Blablacar                0
2896  6797921                   Rennes  ...  Blablacar                0

[4 rows x 6 columns]
 les tickets les plus chers sont:
          id                o_city  ...    company  correspondances
543  6795568  Chalons-En-Champagne  ...  Blablacar                0

[1 rows x 6 columns]
 les tickets au prix médians chers sont:
          id                o_city  ...    company  correspondances
115  6795139           La-Rochelle  ...  Blablacar                0
132  6795156           La-Rochelle  ...  Blablacar                0
557  6795582  Chalons-En-Champagne  ...  Blablacar                0
605  6795630                 

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  app.launch_new_instance()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  app.launch_new_instance()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: UserWarning: Boolean Series

           id    o_city     d_city  price_in_cents  company  correspondances
1873  6796897  Toulouse  Angouleme            1050  Flixbus                2
2587  6797612    Cannes      Nimes            1050   Ouibus                2
 les tickets les plus chers sont:
           id           o_city  ...  company  correspondances
2022  6797047  Aix-En-Provence  ...  Flixbus                2

[1 rows x 6 columns]
 les tickets au prix médians chers sont:
           id    o_city  d_city  price_in_cents   company  correspondances
514   6795539  Bordeaux   Lille            3190   Flixbus                2
520   6795545  Bordeaux   Lille            3190   Flixbus                2
695   6795720      Lyon  Nantes            3190   Flixbus                2
1085  6796110  Besancon  Cannes            3190  Isilines                2
1160  6796185  Besancon  Cannes            3190  Isilines                2
les voyages en train de 200 à 800km :

les tickets les moins chers sont:
         id o_city d_city

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  app.launch_new_instance()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  app.launch_new_instance()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: UserWarning: Boolean Series

Empty DataFrame
Columns: [id, o_city, d_city, price_in_cents, company, correspondances]
Index: []
 les tickets les plus chers sont:
Empty DataFrame
Columns: [id, o_city, d_city, price_in_cents, company, correspondances]
Index: []
 les tickets au prix médians chers sont:
Empty DataFrame
Columns: [id, o_city, d_city, price_in_cents, company, correspondances]
Index: []
2000 - 410000
les voyages en carpooling de 2000 à 410000km :

les tickets les moins chers sont:
Empty DataFrame
Columns: [id, o_city, d_city, price_in_cents, company, correspondances]
Index: []
 les tickets les plus chers sont:
Empty DataFrame
Columns: [id, o_city, d_city, price_in_cents, company, correspondances]
Index: []
 les tickets au prix médians chers sont:
Empty DataFrame
Columns: [id, o_city, d_city, price_in_cents, company, correspondances]
Index: []
les voyages en bus de 2000 à 410000km :

les tickets les moins chers sont:
Empty DataFrame
Columns: [id, o_city, d_city, price_in_cents, company, correspondances]
Ind

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  app.launch_new_instance()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  app.launch_new_instance()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: UserWarning: Boolean Series